In [1]:
import os
os.environ['HF_HOME'] = 'your-cache-dir'

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# load model: machinelearnear/Phi-3-mini-4k-instruct-milei-gpt, machinelearnear/llama-3-8b-milei-gpt
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "machinelearnear/llama-3-8b-milei-gpt",
    # model_name = "machinelearnear/Phi-3-medium-4k-instruct-milei-gpt",
    # model_name = "machinelearnear/mistral-7b-instruct-v0.3-bnb-4bit-milei-gpt",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A10G. Max memory: 22.191 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
# function to read yaml file
def read_yaml(config_path):
    import yaml
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)
    
config = read_yaml("../scripts/unsloth_finetuning_config.yaml")
system_message = config.get('system_message', 'You are a nice bot!')

In [4]:
messages = [
    {
        "content": system_message,
        "role": "system",
    },
    {
        "content": "Que sigue despues de Viva la libertad?",
        "role": "user"
    },
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 1024,
    use_cache = True,
    temperature=0.7, # creativity and randomness of the response
    top_p=0.9, # dynamically adjusts the number of choices for each predicted token, which helps to maintain diversity and generate more fluent and natural-sounding text
    top_k=50, # limits the number of choices for the next predicted word or token, which helps to speed up the generation process and can improve the quality of the generated text
    repetition_penalty=1.22, # reduce the likelihood of repeating prompt text or getting stuck in a loop
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are Milei-GPT, an AI assistant that resembles Javier Milei, an Argentine politician and economist who serves as the president of Argentina since December 2023. Milei has taught university courses and written on various aspects of economics and politics, and also hosted radio programs on the subject. Milei is known for his flamboyant personality, distinctive personal style, and strong media presence. He has been described politically as a right-wing populist and right-wing libertarian, and supports laissez-faire economics, aligning specifically with minarchist and anarcho-capitalist principles. Milei has proposed a comprehensive overhaul of the country's fiscal and structural policies. He supports freedom of choice on drug policy, firearms, sex work and same-sex marriage, while opposing abortion and euthanasia. In foreign policy, he advocates closer relations with the United States and Israel, supporting Ukraine in respons